In [27]:
import numpy as np
from random import Random
import pandas as pd
import gymnasium as gym
from gymnasium import spaces
from seeds import known_seeds
from evaluation import get_actual_demand

In [23]:
space = spaces.Box(low=0, high=1000000, shape=(6, 7, 1), dtype=np.int32)
space.sample()

array([[[389143],
        [562778],
        [601214],
        [ 81962],
        [ 82482],
        [253697],
        [572509]],

       [[973010],
        [157889],
        [346694],
        [674130],
        [ 34021],
        [882586],
        [837218]],

       [[526332],
        [704444],
        [812186],
        [413183],
        [929854],
        [404421],
        [979651]],

       [[ 68894],
        [968565],
        [240875],
        [457410],
        [874471],
        [ 13946],
        [540216]],

       [[517657],
        [595655],
        [426264],
        [420237],
        [276396],
        [809062],
        [453240]],

       [[424046],
        [214966],
        [687868],
        [ 52641],
        [631239],
        [800315],
        [720658]]], dtype=int32)

In [31]:
from gymnasium.wrappers import FlattenObservation # type: ignore
class CustomEnv(gym.Env):
    def __init__(self):
        super(DominoEnv, self).__init__()

        """
        define self.actionspace and self.observation_space below using 
        variables available in "gym.spaces"
        """

        #agent action space (actions it can make)
        ##self.action_space = 
        
        #agent observation space (what the agent can "see"/information that is fed to agent)
        #a 3d array of latency, server_gen, demand, concatenated with 
        #a 3d array of latency, server_gen, supply
        self.observation_space = spaces.Box(low=0, high=1000000, shape=(6, 7, 1), dtype=np.int32)

        self.seeds_array = known_seeds("training")
        self.seed_counter = 0

        self.server_generations = ['CPU.S1', 'CPU.S2', 'CPU.S3', 'CPU.S4', 'GPU.S1', 'GPU.S2', 'GPU.S3']
        self.latencies = ['low', 'medium', 'high']
        self.data_centers = ['DC1', 'DC2', 'DC3', 'DC4']

    #might need func below to convert agent action into a relevant action
    #def conv_agent_action_to_move(self, action):
    
    #returns mask for the action space based on possible plays
    #def valid_action_mask(self):
    
    #initiallise/reset all of the base variables at the end of the "game"
    #has to return a base/initial observation

    def convert_demand_to_observation(self, demand):
        demand_observation = np.zeros((3,7,1))
        for i in len(self.latencies):
            for j in len(self.server_generations):
                demand_observation[i][j][0] = demand.query("latency_sensitivity == {self.latencies[i]}")[self.server_generations[j]]
        return demand_observation

    def convert_fleet_to_observation(self, fleet):
        observed_fleet = np.zeros((3,7,1))
        for i in len(self.data_centers[0:3]):
            for j in len(self.server_generations):
                if(i == "DC3"):
                    gen_total = fleet.query("datacenter_id == {self.datacenters[i]} & server_generation == {self.server_generations[j]}").sum()
                    gen_total += fleet.query("datacenter_id == DC4 & server_generation == {self.server_generations[j]}").sum()
                else:
                    gen_total = fleet.query("datacenter_id == {self.datacenters[i]} & server_generation == {self.server_generations[j]}").sum()
                observed_fleet[i][j][0] = gen_total
        return observed_fleet

    def init_fleet(self):
        fleet = pd.DataFrame()
        for i in self.datacenters:
            for j in self.server_generations:
                fleet["datacenter_id"] = i
                fleet["server_generation"] = j
                fleet["count"] = 0
        return fleet

    def reset(self, seed=None, options=None):
        self.timestep = 0
        self.seed_counter += 1
        self.seed_counter %= 10
        np.random.seed(self.seeds_array[self.seed_counter])

        self.fleet = self.init_fleet()
        self.full_demand = get_actual_demand()
        self.timestep_demand = self.full_demand.query("time_step == {self.timestep}")
        observation_demand = convert_demand_to_observation(self.timestep_demand)
        observation_fleet = convert_fleet_to_observation(self.fleet)
        observation = np.concatenate((observation_demand, observation_fleet))
        self.done = False
        return observation, {}

    #called in a loop where each time it is called the agent chooses an action and change
    #state of game appropriately according to agent action
    def step(self, action):
        """
        after agent move, do it, calc the new observation state
        and the reward from that move it made, if "end" of game set self.done to True

        """
        score = 0
        reward = 0

        #extra info on the game if wanted for yourself
        info = {}
        truncated = False
        
        #observation = 

        return (observation, reward, self.done, truncated, info)
